In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import GridSearchCV

In [4]:
os.chdir("D:\Machine_Hack_Challenge")
df=pd.read_csv("train.csv")

In [5]:
df_test=pd.read_csv("test.csv")

In [6]:
df_test["LOCATION"].unique()

array(['Location 7', 'Location 6', 'Location 8'], dtype=object)

In [7]:
def colNameChanger(df):
    for x in df:
        df=df.rename(columns={str(x):str(x).strip()})
    print(df.info())
    return df
df=colNameChanger(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10276 entries, 0 to 10275
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   NUMBER                       10276 non-null  int64  
 1   DRIVER_NUMBER                10276 non-null  int64  
 2   LAP_NUMBER                   10276 non-null  int64  
 3   LAP_TIME                     10276 non-null  int64  
 4   LAP_IMPROVEMENT              10276 non-null  int64  
 5   CROSSING_FINISH_LINE_IN_PIT  1234 non-null   object 
 6   S1                           10272 non-null  object 
 7   S1_IMPROVEMENT               10276 non-null  int64  
 8   S2                           10268 non-null  object 
 9   S2_IMPROVEMENT               10276 non-null  int64  
 10  S3                           10242 non-null  object 
 11  S3_IMPROVEMENT               10276 non-null  int64  
 12  KPH                          10246 non-null  float64
 13  ELAPSED         

In [8]:
count=0
for x in df["EVENT"]:
    if x.find("Qualifying")==-1:
        count+=1
print(count)

9768


In [9]:
df = df[df["S2"].notna()]

In [10]:
df=df.drop(["DRIVER_NUMBER"],axis="columns")

In [11]:
df_test.describe()

,NUMBER,DRIVER_NUMBER,LAP_NUMBER,LAP_TIME,LAP_IMPROVEMENT,S1_IMPROVEMENT,S2_IMPROVEMENT,S3_IMPROVEMENT,KPH,GROUP,POWER
count,420.000000,420.0,420.000000,0.0,420.000000,420.000000,420.000000,420.000000,418.000000,391.000000,309.0
mean,31.711905,1.0,2.047619,NaN,0.771429,0.771429,0.771429,0.783333,87.079187,2.537084,250.0
std,28.125959,0.0,0.966068,NaN,1.135306,1.135306,1.135306,1.151199,28.960901,1.122283,0.0
min,4.000000,1.0,1.000000,NaN,0.000000,0.000000,0.000000,0.000000,13.900000,1.000000,250.0
25%,10.000000,1.0,1.000000,NaN,0.000000,0.000000,0.000000,0.000000,62.350000,1.500000,250.0
50%,25.000000,1.0,2.000000,NaN,0.000000,0.000000,0.000000,0.000000,91.000000,3.000000,250.0
75%,36.000000,1.0,3.000000,NaN,2.000000,2.000000,2.000000,2.000000,108.875000,4.000000,250.0
max,99.000000,1.0,5.000000,NaN,4.000000,4.000000,4.000000,4.000000,128.000000,4.000000,250.0


In [12]:
df.describe()

,NUMBER,LAP_NUMBER,LAP_TIME,LAP_IMPROVEMENT,S1_IMPROVEMENT,S2_IMPROVEMENT,S3_IMPROVEMENT,KPH,GROUP,POWER
count,10268.000000,10268.000000,10268.000000,10268.000000,10268.000000,10268.000000,10268.000000,10240.000000,507.000000,2903.000000
mean,32.383619,9.681827,92.671504,0.153584,0.152026,0.153974,0.154850,99.869180,2.554241,245.277299
std,28.229251,6.206658,26.215415,0.557303,0.554403,0.558069,0.560527,30.037315,1.122232,6.968022
min,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.400000,1.000000,235.000000
25%,11.000000,4.000000,75.000000,0.000000,0.000000,0.000000,0.000000,89.400000,2.000000,235.000000
50%,25.000000,9.000000,89.000000,0.000000,0.000000,0.000000,0.000000,109.900000,3.000000,250.000000
75%,37.000000,14.000000,103.000000,0.000000,0.000000,0.000000,0.000000,121.400000,4.000000,250.000000
max,99.000000,31.000000,199.000000,4.000000,4.000000,4.000000,4.000000,140.000000,4.000000,250.000000


In [13]:
def missingRemover(df,x):
    if df[str(x)].dtype=="O":
        df[str(x)]=df[str(x)].replace(np.nan,"missing")
        return df
    else:
        df[str(x)]=df[str(x)].replace(np.nan,df[str(x)].median())
        return df

In [14]:
def crossFit(df,x):
    df[str(x)]=df[str(x)].notna().apply(lambda y:1)
    df[str(x)]=df[str(x)].fillna(0)
    return df

In [15]:
def TimeFormatter(df,x):
    df[str(x)]=df[str(x)].fillna("0:0.0")
    df[str(x)]=df[str(x)].apply(
        lambda y:"00:"+str(y) if str(y).find(":")==-1 else y
    )
    df[str(x)]=df[str(x)].apply(
        lambda y:str(y)+".0" if str(y).find(".")==-1 else y
    )
    df[str(x)+"_min"] = pd.to_datetime(df[str(x)], format="%M:%S.%f").dt.minute
    df[str(x)+"_sec"] = pd.to_datetime(df[str(x)], format="%M:%S.%f").dt.second
    df[str(x)+"_micro"] = pd.to_datetime(df[str(x)], format="%M:%S.%f").dt.microsecond
    df=df.drop([str(x)],axis="columns")
    return df

In [16]:
def powerMap(df,x):
    df[str(x)]=df[str(x)].replace(np.nan,200)
    df=df[df[str(x)] != 235]
    return df

In [17]:
def kphEstimator(df,x,y):
    df[str(x)]=df.groupby(str(y))[str(x)].apply(lambda y:y.fillna(y.mean()))
    return df

In [18]:
def Group(df,x):
    df[str(x)]=df[str(x)].fillna(0)
    return df

In [19]:
df=crossFit(df,"PIT_TIME")
df=crossFit(df,"CROSSING_FINISH_LINE_IN_PIT")
df=kphEstimator(df,"KPH","NUMBER")
df=powerMap(df,"POWER")
df=TimeFormatter(df,"S3")
df=TimeFormatter(df,"S2")
df=TimeFormatter(df,"S1")
df=TimeFormatter(df,"S1_LARGE")
df=TimeFormatter(df,"S2_LARGE")
df=TimeFormatter(df,"S3_LARGE")
df=Group(df,"GROUP")
df=TimeFormatter(df,"HOUR")
df=TimeFormatter(df,"ELAPSED")

In [20]:
df_test=colNameChanger(df_test)
df_test=crossFit(df_test,"PIT_TIME")
df_test=crossFit(df_test,"CROSSING_FINISH_LINE_IN_PIT")
df_test=kphEstimator(df_test,"KPH","NUMBER")
df_test=powerMap(df_test,"POWER")
df_test=TimeFormatter(df_test,"S3")
df_test=TimeFormatter(df_test,"S2")
df_test=TimeFormatter(df_test,"S1")
df_test=TimeFormatter(df_test,"S1_LARGE")
df_test=TimeFormatter(df_test,"S2_LARGE")
df_test=TimeFormatter(df_test,"S3_LARGE")
df_test=Group(df_test,"GROUP")
df_test=TimeFormatter(df_test,"HOUR")
df_test=TimeFormatter(df_test,"ELAPSED")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   NUMBER                       420 non-null    int64  
 1   DRIVER_NUMBER                420 non-null    int64  
 2   LAP_NUMBER                   420 non-null    int64  
 3   LAP_TIME                     0 non-null      float64
 4   LAP_IMPROVEMENT              420 non-null    int64  
 5   CROSSING_FINISH_LINE_IN_PIT  16 non-null     object 
 6   S1                           420 non-null    object 
 7   S1_IMPROVEMENT               420 non-null    int64  
 8   S2                           420 non-null    object 
 9   S2_IMPROVEMENT               420 non-null    int64  
 10  S3                           418 non-null    object 
 11  S3_IMPROVEMENT               420 non-null    int64  
 12  KPH                          418 non-null    float64
 13  ELAPSED             

In [21]:
# df=kphEstimator(df,"KPH","NUMBER")

In [22]:
# df=powerMap(df,"POWER")

In [23]:
# df=TimeFormatter(df,"S3")

In [24]:
# df=TimeFormatter(df,"S2")

In [25]:
# df=TimeFormatter(df,"S1")

In [26]:
# df=TimeFormatter(df,"S1_LARGE")

In [27]:
# df=TimeFormatter(df,"S2_LARGE")

In [28]:
# df=TimeFormatter(df,"S3_LARGE")

In [29]:
# df=Group(df,"GROUP")

In [30]:
# df=TimeFormatter(df,"HOUR")

In [31]:
# df=TimeFormatter(df,"ELAPSED")

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9354 entries, 0 to 10275
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   NUMBER                       9354 non-null   int64  
 1   LAP_NUMBER                   9354 non-null   int64  
 2   LAP_TIME                     9354 non-null   int64  
 3   LAP_IMPROVEMENT              9354 non-null   int64  
 4   CROSSING_FINISH_LINE_IN_PIT  9354 non-null   int64  
 5   S1_IMPROVEMENT               9354 non-null   int64  
 6   S2_IMPROVEMENT               9354 non-null   int64  
 7   S3_IMPROVEMENT               9354 non-null   int64  
 8   KPH                          9354 non-null   float64
 9   DRIVER_NAME                  9354 non-null   object 
 10  PIT_TIME                     9354 non-null   int64  
 11  GROUP                        9354 non-null   float64
 12  TEAM                         9354 non-null   object 
 13  POWER            

In [33]:
df["NUMBER"].dtype=="O"

False

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9354 entries, 0 to 10275
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   NUMBER                       9354 non-null   int64  
 1   LAP_NUMBER                   9354 non-null   int64  
 2   LAP_TIME                     9354 non-null   int64  
 3   LAP_IMPROVEMENT              9354 non-null   int64  
 4   CROSSING_FINISH_LINE_IN_PIT  9354 non-null   int64  
 5   S1_IMPROVEMENT               9354 non-null   int64  
 6   S2_IMPROVEMENT               9354 non-null   int64  
 7   S3_IMPROVEMENT               9354 non-null   int64  
 8   KPH                          9354 non-null   float64
 9   DRIVER_NAME                  9354 non-null   object 
 10  PIT_TIME                     9354 non-null   int64  
 11  GROUP                        9354 non-null   float64
 12  TEAM                         9354 non-null   object 
 13  POWER            

In [35]:
weather_data=pd.read_csv("train_weather.csv")
test_weather_data=pd.read_csv("test_weather.csv")

In [36]:
weather_data

,TIME_UTC_SECONDS,TIME_UTC_STR,AIR_TEMP,TRACK_TEMP,HUMIDITY,PRESSURE,WIND_SPEED,WIND_DIRECTION,RAIN,LOCATION,EVENT
0,1618042506,10/04/21 8:15,15,18.4,60,1018.05,2.12187,165,-1,Location 2,Free Practice 2
1,1618042567,10/04/21 8:16,15.0556,18.4,60,1018.12,1.06093,177,-1,Location 2,Free Practice 2
2,1618042632,10/04/21 8:17,15.0556,18.5,60,1018.12,2.12187,157,-1,Location 2,Free Practice 2
3,1618042697,10/04/21 8:18,15.0556,18.5,60,1018.08,2.12187,163,-1,Location 2,Free Practice 2
4,1618042762,10/04/21 8:19,15.1111,18.5,60,1018.15,3.1828,149,-1,Location 2,Free Practice 2
...,...,...,...,...,...,...,...,...,...,...,...
909,1625936875,10/07/21 17:07,"2,56,111","35,9",75,"1012,84","31,828",324,-1,Location 6,Qualifying Group 4
910,1625936173,10/07/21 16:56,"2,55,556","35,3",76,"1012,97","1,06,093",5,-1,Location 6,Qualifying Group 3
911,1625936234,10/07/21 16:57,"2,55,556","35,4",76,"1012,9","2,12,187",5,-1,Location 6,Qualifying Group 3
912,1625936299,10/07/21 16:58,"2,55,556","35,8",76,"1012,87","1,06,093",328,-1,Location 6,Qualifying Group 3


In [37]:
weather_data=weather_data.drop(["TIME_UTC_SECONDS","TIME_UTC_STR","RAIN"],axis="columns")
test_weather_data=test_weather_data.drop(["TIME_UTC_SECONDS","TIME_UTC_STR","RAIN"],axis="columns")

In [38]:
def changeDatatypes(df,lis):
    for x in lis:
        df[str(x)]=df[str(x)].replace(",", ".",regex=True)
        df[str(x)]=pd.to_numeric(df[str(x)],errors="coerce")
        df[str(x)]=df.groupby("LOCATION")[str(x)].apply(lambda y:y.fillna(y.mean()))

    return df

In [39]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 914 entries, 0 to 913
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   AIR_TEMP        914 non-null    object
 1   TRACK_TEMP      914 non-null    object
 2   HUMIDITY        914 non-null    object
 3   PRESSURE        914 non-null    object
 4   WIND_SPEED      914 non-null    object
 5   WIND_DIRECTION  914 non-null    int64 
 6   LOCATION        914 non-null    object
 7   EVENT           914 non-null    object
dtypes: int64(1), object(7)
memory usage: 57.2+ KB


In [40]:
# quantitative_fields=["AIR_TEMP","TRACK_TEMP","HUMIDITY","WIND_SPEED","WIND_DIRECTION","RAIN"] track temp,humidity,wind direction
quantitative_fields=["PRESSURE","AIR_TEMP","TRACK_TEMP","HUMIDITY","WIND_SPEED","WIND_DIRECTION"]

In [41]:
len(quantitative_fields)

6

In [42]:
weather_data=changeDatatypes(weather_data,quantitative_fields)
test_weather_data=changeDatatypes(test_weather_data,quantitative_fields)

In [43]:
test_weather_data

,AIR_TEMP,TRACK_TEMP,HUMIDITY,PRESSURE,WIND_SPEED,WIND_DIRECTION,LOCATION,EVENT
0,20.35,25.4,63.94,1016.46,1.18,203,Location 8,Free Practice 2
1,20.33,25.4,63.96,1016.48,10.83,204,Location 8,Free Practice 2
2,20.25,25.4,64.04,1016.48,12.53,201,Location 8,Free Practice 2
3,20.24,25.3,64.22,1016.49,2.28,198,Location 8,Free Practice 2
4,20.27,25.1,64.14,1016.50,11.28,204,Location 8,Free Practice 2
...,...,...,...,...,...,...,...,...
162,24.06,34.7,46.03,1010.14,9.11,204,Location 8,Qualifying Group 4
163,24.19,35.4,46.33,1010.38,6.18,208,Location 8,Qualifying Group 3
164,24.07,35.4,46.18,1010.33,5.95,214,Location 8,Qualifying Group 3
165,24.03,35.3,45.93,1010.32,9.53,219,Location 8,Qualifying Group 3


In [44]:
def groupbyOrganizer(weather_data,quantitative_fields):
    grouped_data=[]

    for x in quantitative_fields:
        grouped_data.append(weather_data.groupby(["LOCATION","EVENT"])[x].mean().to_frame().reset_index())
    
    weather_data=weather_data[["LOCATION","EVENT"]]
    for i in range(0,len(quantitative_fields)):
        weather_data=weather_data.merge(grouped_data[i], how = "right", on = ["LOCATION","EVENT"])
        weather_data.drop_duplicates(keep="first", inplace=True)
    return weather_data.reset_index(drop=True)
    #return grouped_data
#merged_df = DF2.merge(DF1, how = 'inner', on = ['date', 'hours'])
new_weather=groupbyOrganizer(weather_data,quantitative_fields)
new_test_weather=groupbyOrganizer(test_weather_data,quantitative_fields)

In [45]:
df_new=df.merge(new_weather, how = "inner", on = ["LOCATION","EVENT"])
df_test_new=df_test.merge(new_weather, how = "left", on = ["LOCATION","EVENT"])

In [46]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7205 entries, 0 to 7204
Data columns (total 46 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   NUMBER                       7205 non-null   int64  
 1   LAP_NUMBER                   7205 non-null   int64  
 2   LAP_TIME                     7205 non-null   int64  
 3   LAP_IMPROVEMENT              7205 non-null   int64  
 4   CROSSING_FINISH_LINE_IN_PIT  7205 non-null   int64  
 5   S1_IMPROVEMENT               7205 non-null   int64  
 6   S2_IMPROVEMENT               7205 non-null   int64  
 7   S3_IMPROVEMENT               7205 non-null   int64  
 8   KPH                          7205 non-null   float64
 9   DRIVER_NAME                  7205 non-null   object 
 10  PIT_TIME                     7205 non-null   int64  
 11  GROUP                        7205 non-null   float64
 12  TEAM                         7205 non-null   object 
 13  POWER             

In [47]:
df_new["EVENT"].unique()

array(['Free Practice 2', 'Qualifying Group 1', 'Free Practice 1',
       'Qualifying Group 2', 'Qualifying Group 4', 'Qualifying Group 3',
       'Free Practice 3'], dtype=object)

In [48]:
df_test_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420 entries, 0 to 419
Data columns (total 47 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   NUMBER                       420 non-null    int64  
 1   DRIVER_NUMBER                420 non-null    int64  
 2   LAP_NUMBER                   420 non-null    int64  
 3   LAP_TIME                     0 non-null      float64
 4   LAP_IMPROVEMENT              420 non-null    int64  
 5   CROSSING_FINISH_LINE_IN_PIT  420 non-null    int64  
 6   S1_IMPROVEMENT               420 non-null    int64  
 7   S2_IMPROVEMENT               420 non-null    int64  
 8   S3_IMPROVEMENT               420 non-null    int64  
 9   KPH                          420 non-null    float64
 10  DRIVER_NAME                  420 non-null    object 
 11  PIT_TIME                     420 non-null    int64  
 12  GROUP                        420 non-null    float64
 13  TEAM                

In [49]:
team_df=pd.get_dummies(df_new["TEAM"],drop_first=True)

In [50]:
team_df

,AR,BMWE,BMWl,DRAGONR,DRAGONo,DRAGON√É,DSo,DS¬â,EA,EF,...,MAHINDRAd,ME,Ml,N,NIO,NIOO,Nt,ROK,ROKo,TAG
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7200,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7202,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7203,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [51]:
def finalProcedure(df_new,team,driver):
#     df_new=df_new.drop(["LOCATION","EVENT"],axis="columns")
    team_df=pd.get_dummies(df_new[str(team)],drop_first=True)
    driver_df=pd.get_dummies(df_new[str(driver)],drop_first=True)
    df_new=df_new.drop(["LOCATION","EVENT",str(team),str(driver)],axis="columns")
    df_new=pd.concat([df_new,team_df],axis=1)
    df_new=pd.concat([df_new,driver_df],axis=1)
    return df_new
#     df_new=df_new.drop([str(x)],axis="columns")
#     df_new=pd.concat([df_new,team_df],axis=0)
#     df = pd.get_dummies(df,drop_first=True)

In [52]:
df_new=finalProcedure(df_new,"TEAM","DRIVER_NAME")
df_test_new=finalProcedure(df_test_new,"TEAM","DRIVER_NAME")
df_test_new=df_test_new.fillna(df_test_new.median())

In [53]:
df_test_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420 entries, 0 to 419
Data columns (total 86 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   NUMBER                       420 non-null    int64  
 1   DRIVER_NUMBER                420 non-null    int64  
 2   LAP_NUMBER                   420 non-null    int64  
 3   LAP_TIME                     0 non-null      float64
 4   LAP_IMPROVEMENT              420 non-null    int64  
 5   CROSSING_FINISH_LINE_IN_PIT  420 non-null    int64  
 6   S1_IMPROVEMENT               420 non-null    int64  
 7   S2_IMPROVEMENT               420 non-null    int64  
 8   S3_IMPROVEMENT               420 non-null    int64  
 9   KPH                          420 non-null    float64
 10  PIT_TIME                     420 non-null    int64  
 11  GROUP                        420 non-null    float64
 12  POWER                        420 non-null    float64
 13  S3_min              

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [55]:
inputs=df_new.drop(["LAP_TIME","N¬úL","DRAGON√É"],axis="columns")
inputs_test=df_test_new.drop(["LAP_TIME"],axis="columns")
outputs=df_new["LAP_TIME"]

In [56]:
inputs_test=inputs_test.drop(["DRIVER_NUMBER"],axis="columns")

In [57]:
x_train, x_test, y_train, y_test = train_test_split(inputs,outputs,train_size=0.2)

In [ ]:
param_grid = {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100,300,500,1000,2000]
    }
clf = GridSearchCV(LogisticRegression(), param_grid, verbose=True, n_jobs=-1)
clf.fit(x_train,y_train)
print(clf.best_params_)

Fitting 5 folds for each of 2000 candidates, totalling 10000 fits


C:\ProgramData\Anaconda3\envs\rstudio\lib\site-packages\sklearn\model_selection\_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [160]:

model =LogisticRegression(C=1.623776739188721, max_iter=100, penalty="l1",solver="saga")
model.fit(x_train, y_train)
y_pred=model.predict(x_test)

C:\ProgramData\Anaconda3\envs\rstudio\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [ ]:
len(x_train)

In [ ]:
len(x_test)

In [ ]:
len(y_train)

In [ ]:
len(y_test)

In [ ]:
y_pred=model.predict_proba(x_test)

In [ ]:
type(y_test)

In [ ]:
y_pred

In [68]:
from sklearn.metrics import mean_squared_log_error
np.sqrt(mean_squared_log_error(y_test,y_pred)) 

0.3674260913582369

In [69]:
# from sklearn.ensemble import RandomForestRegressor
# model = RandomForestRegressor(n_estimators=5000,max_depth=70,max_features="auto",min_samples_leaf=4,min_samples_split=10)
# model.fit(x_train, y_train)
# y_pred=model.predict(x_test)
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=5000,max_depth=70,max_features="auto",min_samples_leaf=4,min_samples_split=10)
model.fit(inputs, outputs)
# y_pred=model.predict(x_test)

RandomForestRegressor(max_depth=70, min_samples_leaf=4, min_samples_split=10,
                      n_estimators=5000)

In [ ]:
# from sklearn.svm import SVR
# model=SVR(C=1000,gamma=0.001,kernel="sigmoid")
# model.fit(x_train,y_train)
# y_pred=model.predict(x_test)

In [132]:
# model.fit(x_train,y_train)
# y_pred=model.predict(x_test)

In [117]:
l1=[x for x in inputs]
l2=[x for x in inputs_test]

In [118]:
res = list(set(l1)^set(l2))

In [170]:
y_pred_test=model.predict(inputs_test)

In [73]:
np.unique(y_pred_test)

array([ 94.05624 ,  94.18098 ,  94.41131 ,  94.77171 ,  94.811   ,
        94.97841 ,  94.984604,  95.092575,  95.21545 ,  95.232895,
        95.29658 ,  95.310814,  95.32752 ,  95.36039 ,  95.36774 ,
        95.40258 ,  95.42086 ,  95.430244,  95.475716,  95.4867  ,
        95.51408 ,  95.52331 ,  95.54822 ,  95.56627 ,  95.56927 ,
        95.57857 ,  95.581505,  95.61293 ,  95.646324,  95.665306,
        95.66805 ,  95.703224,  95.709335,  95.7469  ,  95.75434 ,
        95.75809 ,  95.75971 ,  95.768875,  95.78636 ,  95.80746 ,
        95.82573 ,  95.82725 ,  95.847916,  95.854416,  95.85916 ,
        95.87159 ,  95.87552 ,  95.88425 ,  95.89341 ,  95.9072  ,
        95.917984,  95.9297  ,  95.93376 ,  95.93661 ,  95.950096,
        95.96057 ,  95.960594,  95.9641  ,  95.96569 ,  95.97156 ,
        95.97163 ,  95.97454 ,  95.98855 ,  95.99669 ,  96.018524,
        96.03896 ,  96.05422 ,  96.05892 ,  96.06225 ,  96.0732  ,
        96.09226 ,  96.09709 ,  96.09883 ,  96.10352 ,  96.104

In [140]:
len(y_pred_test)

420

In [81]:
submission_df=pd.DataFrame()

In [82]:
submission_df["LAP_TIME"]=y_pred_test

In [83]:
submission_df

,LAP_TIME
0,97.905197
1,98.135979
2,97.506516
3,97.670959
4,106.739380
...,...
415,97.995201
416,98.038651
417,96.569229
418,95.726089


In [84]:
submission_df.to_csv('latest_submission_file.csv', index=False)

In [2]:
pip install xgboost

In [78]:
import xgboost
from xgboost import XGBRFRegressor
model = XGBRFRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)


In [79]:
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [71]:
model.fit(inputs,outputs)

XGBRFRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bytree=0.8, enable_categorical=False, eta=0.1, gamma=0,
               gpu_id=-1, importance_type=None, interaction_constraints='',
               max_delta_step=0, max_depth=7, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=1000, n_jobs=8,
               num_parallel_tree=1000, objective='reg:squarederror',
               predictor='auto', random_state=0, reg_alpha=0,
               scale_pos_weight=1, subsample=0.7, tree_method='exact',
               validate_parameters=1, verbosity=None)

In [80]:
y_pred_test=model.predict(inputs_test)